# 12. Exception Handling

## Advanced Exception Handling
---

### Nested `Exception`s

  
Every `Exception` could have a **nested exception**, also sometimes called an **inner exception**, **wrapped exception**, or **internal exception**.   
   
The ability to wrap an exception with another exception is very useful in some cases and allows exceptions to be linked in the so called **exception chain**.

<br>

In [1]:
// provides the TextReader and StreamReader clasees
using System.IO;

<br>

Let's suppose we have a **software component** (let’s call it **Component** **A**):   

In [2]:
public void ComponentA( string badFileName )
{ 

    // Code that potentially results in an Exception 

    TextReader reader = new StreamReader( badFileName ); 
    string line = reader.ReadLine(); 
    Console.WriteLine(line); 
    reader.Close();

}

<br>

This **Component A**, above,  is then internally used by another **component** (called **Component** **B**). 

In [3]:
public void ComponentB( )
{ 

    // A Bad File Name, with which we intend to test
    // how our program will respond to unexpected inputs:
    string badFileName = "1 will definately%20cause an Exception.pdffff";


    // Add code within the try block that potentially results in an Exception 
    try 
    {
        ComponentA( badFileName );
    }


    // Implement our own handler for the  FileNotFoundException
    // which prints out a meessage explaining the issue
    catch( Exception e )
    {
        Console.WriteLine( 
            $"Oh SNAP! Here's what went wrong in Component B:\n{ e }" 
        );
    }
    
}

<br>

When **B** throws a **B** **exception** (an **exception** defined in **B**),   
**A** will have to *propagate the error*, since it will interfere with any subsequent handler operations.   

In [4]:
ComponentB();

Oh SNAP! Here's what went wrong in Component B:
System.IO.FileNotFoundException: Could not find file 'c:\Users\harry\Dev\projects\Notes On C-Sharp\12. Exception Handling\1 will definately%20cause an Exception.pdffff'.
File name: 'c:\Users\harry\Dev\projects\Notes On C-Sharp\12. Exception Handling\1 will definately%20cause an Exception.pdffff'
   at Microsoft.Win32.SafeHandles.SafeFileHandle.CreateFile(String fullPath, FileMode mode, FileAccess access, FileShare share, FileOptions options)
   at Microsoft.Win32.SafeHandles.SafeFileHandle.Open(String fullPath, FileMode mode, FileAccess access, FileShare share, FileOptions options, Int64 preallocationSize, Nullable`1 unixCreateMode)
   at System.IO.Strategies.OSFileStreamStrategy..ctor(String path, FileMode mode, FileAccess access, FileShare share, FileOptions options, Int64 preallocationSize, Nullable`1 unixCreateMode)
   at System.IO.Strategies.FileStreamHelpers.ChooseStrategyCore(String path, FileMode mode, FileAccess access, FileShare

Because **B** cannot simply throw an **A**-**exception**,    
it must throw an **B**-**exception** *containing* the **A**-**exception** as a **nested** **exception**.

<br>

### Which Exceptions to Handle and Which Not?

There is *one universal rule* regarding **exception handling**:

> A method should only handle exceptions which it *expects* and which it *knows how to process*.<br>All the other exceptions must be left to the calling method.

If we follow this rule and eventually we reach the `Main()` method (or the starting method of the respective thread of execution), if the exception is not yet caught, then the $CLR$ itself will display the error.

A method is considered as *competent to handle an* **exception** if it *expects* this **exception**, and it has the information *why the* **exception** *has been thrown* and *what to do* in this situation.    
   
If we have a method that must *read a text file* and return its contents as a string, that method might catch `FileNotFoundException` and return an empty string in this case.   
   
Still, this same method will hardly be able to correctly handle `OutOfMemoryException`.   
   
What should the method do in case of insufficient memory? Return an empty string? Throw some other exception? Do something completely different? 
   
So apparently the method is not competent to handle such exception and thus the best way is to pass the exception up to the calling method so it could (hopefully) be handled at some other level by a method competent to do it. 
   
Using this simple philosophy allows exception handling to be done in a structured and systematic way.

<br>

### The `try`-`finally` Construct

Every `try` **block** could contain a subsequent `finally` **block**. 
   
The code within the `finally` **block** is *always executed*, no matter how the program flow leaves the `try` **block**.    
   
This guarantees that the `finally` **block** will be executed even if an exception is thrown or a return statement is executed within the `try` **block**.

<br>

The basic form of the `finally` **block** is given as follows:
```c#
try
{
    // Some code that could potentially cause an exception
}

finally
{
    // Some code that will execute no matter if an exception occurs or not.
}
```

<br>

Every `try` **block** may have zero or more `catch` **block**s and at most one `finally` **block**.    
It is possible to have multiple `catch` **block**s and a `finally` **block** in the same `try`-`catch`-`finally` construct:

```c#
try
{
    // Some code that could potentially cause an exception
}

catch ( Exception someException ) 
{
    // Some code handling some exception
}

catch ( Exception someOtherException ) 
{
    // Some code handling some other exception
}

finally
{
    // Some code that will execute no matter if an exception occurs or not.
}
```

<br>

#### When Should We Use `try`-`finally`?

There's many cases when we have to work with $external\,resources$ for our programs.      
Examples for $external\,resources$ include: 
- *files*
- *network connections*
- *graphical elements*
- *pipes* and *streams to or from different hardware devices*
  - *printers*
  - *card readers*   
 
When we deal with such $external\,resource$, it is critically important to **free up the resources** as early as possible when the resource is no longer needed.   

For example, when we *open a file to read its contents* (let’s say to load a JPG image), we must **close the file** right after we have read the contents.    
   
If we leave the file open, the operating system will prevent other users and applications to make certain operations on the file. Perhaps you faced such a situation when you could not delete some directory or a file because it is being used by a running process.   
                      
The `finally` **block** is priceless when we need to **free an external resource** or **make any other cleanup**.    
   
The `finally` **block** guarantees that the **cleanup operations** will not be accidentally skipped because of an unexpected exception or because of execution of `return`, `continue` or `break` statements.    

<br>

#####  Resource Cleanup – Defining the Problem

Because proper **resource management** is an important concept in programming, we will look at it in some more details.

In the above example, when we wanted to **read a file**, we exectuted the following code *to create an instance of the* `StreamReader()` *class*, which enable us to will *store the data taken in from the reader*, and then subsequently *close the data stream*.

Recall that we implemented the example above as follows:

```c#
public void ComponentA( string badFileName )
{ 

    // Code that potentially results in an Exception 

    TextReader reader = new StreamReader( badFileName ); 
    string line = reader.ReadLine(); 
    Console.WriteLine(line); 
    reader.Close();

}
```

So **what is the problem with this code?**,   
Well, what the code is supposed to *do* is: 
1. *Open* up a file reader
2. *Read* the data
3. *Close* the reader before the method returns. 

The issue is that, potentially, the method could finish in one of several ways: 
- An exception could be thrown when the reader is initialized (say if the file is missing).
- During reading the file, an exception could arise (imagine a file on a remote network device which goes offline during file reading).

<br>

#####  Resource Cleanup – Solving the Problem

So now that we are aware of the inherent flaw of the plain old $Open \Rrightarrow Read \Rrightarrow Close$ approach,   
how can we go about solving the problem?  

This is actually a conerstone use case for the `try`-`finally` constuct:

In [5]:
void ProperlyErrorHandled( string potentiallyBadFilename )
{
    
    // If an Exception happens here,  
    // the file will never be opened to begin with
    TextReader reader =  new StreamReader( potentiallyBadFilename );


    // Execute code that will potentially throw an Exeption
    try
    {
        string line = reader.ReadLine();
        Console.WriteLine( line );
    }


    // This will close the file no matter if the code
    // in the try block throws an Exception or not
    finally
    {
        reader.Close();
    }

}

This code is cleaner, shorter and clearer, and is known as a **dispose pattern**.    
However, note that this way, any resulting Exception will go up to the method calling `ProperlyErrorHandled(…)`.

<br>

##### Cleaning Up Multiple Resources

Sometimes we need to **free more than one resource**.    
It can be a good practice to free the resources in the *reverse order* that they were allocated in:

```c#
void ProperlyErrorHandledAndNested( someType potentiallyBadData )
{
    
    // If an Exception happens here,  
    // Resource 2 will never get allocated to begin with
    Resource r1 =  new Resource();
    
    // Execute code that will potentially throw an Exeption
    try
    {
        Resource r2 = new Resource2();

        try
        {
            // use r1 and r2
        }

        // This will clean up Resource 2 no matter if the code
        // in the try block throws an Exception or not
        finally
        {
            r2.Release();
        }

    }

    // This will clean up Resource 1 no matter if the code
    // in the try block throws an Exception or not
    finally
    {
        r1.Release();
    }

}
```

<br>

Another option is to *declare all of the resources in advance*,  
and then make the cleanup in a single `finally` **block** with respective `null` checks:

```c#
void AnotherProperlyErrorHandledAndNested( someType potentiallyBadData )
{
    
    // Declare both Resources in advance and initialize them as null values 
    Resource r1 = null; 
    Resource r2 = null; 
    
    // Execute code that will potentially throw an Exeption
    try 
    {
        // Allocate the memory for both Resources
        Resource r1 = new Resource1(); 
        Resource r2 = new Resource2();

        // Use both r1 and r2
    } 
    
    // This will clean up the Resources no matter if the code
    // in the try block throws an Exception or not,
    finally 
    {
        if (r1 != null) 
            r1.Release();

        if (r2 != null) 
            r2.Release();
    }
}
```

<br>

### The `IDisposable` and the `using` Statement

There are other programming contructs which *shorten* and *simnplify* the above mentioned techniques for **releasing resources** in $C\#$.

#### `IDisposable`

The main use of `IDisposable` **interface** is to **release resources**.   
In $.NET$, examples of such resources are *window handles*, *files*, *streams*, and others.

One important method of the `IDisposable` interface is` Dispose()`.    
The main thing we need to know about the method is that it **releases the resources of the class that implements it**.   

```c#
StreamReader reader = new StreamReader(fileName); 
try 
{
// Use the reader here
} 

finally 
{
    if (reader != null) 
        reader.Dispose(); 
}
```

In cases when resources are *streams*, *readers* or *files*, **releasing resources** can be done using the `Dispose()` method from `IDisposable` **interface**, which calls their `Close()` method.     
   
This method **closes them** and **releases their resources**.

<br>

#### The `using` Keyword

The example above can be written in a shorter form with the help of the `using` keyword:

```c#
using ( StreamReader reader = new StreamReader( fileName ) )
{
    // Use the reader here
}
```

The above simplified form of the **dispose pattern** is simple to write, simple to use, simple to read, and is guaranteed to correctly **release the allocated resources** specified in the brackets of the `using` statement.
  
Here, it is not necessary to have `try`-`finally`,    
or to explicitly call any method in order to **release the resources**.
     
The compiler takes care to automatically put `try`-`finally` **block** and the used resources are released by calling the `Dispose()` method after leaving the `using` **block**.   

<br>

##### Nested `using` Statements

`using` **statements** can be **nested** one within another:

```
using ( ResourceType r1 = ... )
    using ( ResourceType r2 = ... )
     ...
        using ( ResourceType rN = ...   )
            statements;
```

But the figure above merely illustrates the concept,    
the actual $C\#$ implementation is generalized as follows:

```c#
using ( ResourceType r1 = …, r2 = …, …, rN = …) 
{
    // statements 
}
```

<br>

It is important to mention that the `using` statement is not related to exception handling.   
   
Its only purpose is to **release the resources** no matter whether exceptions are thrown or not. It does not handle exception.

<br>

##### When to Use the `using` Statement?

There is a simple rule when to use `using` with $.NET$ classes:

> Use the `using` **statement** with all classes that implement the `IDisposable` **interface**. 

When a class implements `IDisposable` **interface**, this means that the creator of this class expects it can be used with the `using` **statement**, and the class contains some expensive resource that should not be left unreleased.    
   
Implementing `IDisposable` also means that it should be released immediately after we finish using the class, and the easiest way to do this in $C\#$ is with the  `using` **statement**.